In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# 是否将数量的负值认为是退款
allow_refund = True
time_parse = lambda x:datetime.strptime(x,"%d-%m-%Y %H:%M")
user_data = pd.read_csv('./data/OnlineRetail.csv',parse_dates=['InvoiceDate'],date_parser=time_parse)
user_data = user_data.dropna(how='any',axis=0)
# 筛选出数量大于0 价格大于等于0的条目索引
index_valid_quantity = user_data['Quantity']>0 
index_valid_price = user_data['UnitPrice']>=0
user_data['CustomerID'] = user_data['CustomerID'].astype(np.int64)
if allow_refund == True:
    index_valid = index_valid_price
else:
    index_valid = [all(x) for x in zip(index_valid_quantity,index_valid_price)]
# 清洗数据取出有效的数据
user_data = user_data[index_valid]
user_data.reset_index(drop=True,inplace=True)
# 派生订单总额列 将数值设置为单价与数量的乘积
user_data['orderAmount'] = user_data['Quantity'] * user_data['UnitPrice']
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406829 entries, 0 to 406828
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   CustomerID   406829 non-null  int64         
 1   Quantity     406829 non-null  int64         
 2   InvoiceDate  406829 non-null  datetime64[ns]
 3   UnitPrice    406829 non-null  float64       
 4   orderAmount  406829 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 15.5 MB


In [3]:
# 将用户ID作为索引创建透视表
user_pivot = user_data.pivot_table(index='CustomerID',aggfunc={
    'Quantity':'sum',
    'InvoiceDate':'max',
    'orderAmount':'sum'
})
# 取出日期中的最大值
date_now = user_pivot['InvoiceDate'].max()
# 计算日期中最大值与日期平均值的差
date_gap_mean = date_now - user_pivot['InvoiceDate'].mean()
# 计算购买数量的平均值
quantity_mean = user_pivot['Quantity'].mean()
# 计算购买总额的平均值
order_amount_mean = user_pivot['orderAmount'].mean()
# 将数据中数值与对应列的平均值做差作为M（总额）F（频率）R（最近购买）的数值
user_pivot['M'] = user_pivot['orderAmount'] - order_amount_mean
user_pivot['F'] = user_pivot['Quantity'] - quantity_mean
user_pivot['R'] = (date_now - user_pivot['InvoiceDate'] - date_gap_mean)/np.timedelta64(1,'D')
user_pivot

,InvoiceDate,Quantity,orderAmount,M,F,R
CustomerID,,,,,,
12346,2011-01-18 10:17:00,0,0.00,-1898.459701,-1122.344007,233.538272
12347,2011-12-07 15:52:00,2458,4310.00,2411.540299,1335.655993,-89.694367
12348,2011-09-25 13:13:00,2341,1797.24,-101.219701,1218.655993,-16.583950
12349,2011-11-21 09:51:00,631,1757.55,-140.909701,-491.344007,-73.443672
12350,2011-02-02 16:01:00,197,334.40,-1564.059701,-925.344007,218.299383
...,...,...,...,...,...,...
18280,2011-03-07 09:52:00,45,180.60,-1717.859701,-1077.344007,185.555633
18281,2011-06-12 10:53:00,54,80.82,-1817.639701,-1068.344007,88.513272
18282,2011-12-02 11:43:00,98,176.60,-1721.859701,-1024.344007,-84.521450


In [4]:
#　RFM字符串与用户标签映射关系
value_map = {
    '111':'重要价值客户',
    '011':'重要保持客户',
    '101':'重要挽留客户',
    '001':'重要发展客户',
    '110':'一般价值客户',
    '010':'一般保持客户',
    '100':'一般挽留客户',
    '000':'一般发展客户'
}
# RFM的值大于等于0则映射到1，否则为0，将RFM字符串拼接，并通过映射关系得到用户标签
def add_label(x):
    level = x.map(lambda x : '1' if x >= 0 else '0')
    label = level.R + level.F + level.M
    return value_map[label]
RFM = user_pivot[['R','F','M']]
RFM['label'] = RFM.apply(add_label,axis=1)
RFM

D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,R,F,M,label
CustomerID,,,,
12346,233.538272,-1122.344007,-1898.459701,一般挽留客户
12347,-89.694367,1335.655993,2411.540299,重要保持客户
12348,-16.583950,1218.655993,-101.219701,一般保持客户
12349,-73.443672,-491.344007,-140.909701,一般发展客户
12350,218.299383,-925.344007,-1564.059701,一般挽留客户
...,...,...,...,...
18280,185.555633,-1077.344007,-1717.859701,一般挽留客户
18281,88.513272,-1068.344007,-1817.639701,一般挽留客户
18282,-84.521450,-1024.344007,-1721.859701,一般发展客户


In [7]:
RFM.groupby("label").size()

label
一般价值客户      22
一般保持客户     131
一般发展客户    1945
一般挽留客户    1353
重要价值客户      53
重要保持客户     742
重要发展客户     109
重要挽留客户      17
dtype: int64